In [1]:
#Fill missing latitude/longitude with country averages 
import pandas as pd 

df = pd.read_csv("EuCitiesTemperatures.csv")

#fill missing lat/lon with country averages (rounded to 2 decimals)
df['latitude'] = df.groupby('country')['latitude'].transform(lambda x: x.fillna(round(x.mean(), 2)))

print(df[['city', 'country', 'latitude', 'longitude']].head())

        city  country  latitude  longitude
0    Elbasan  Albania     41.12      20.08
1    Andorra  Andorra     42.50       1.52
2  Innsbruck  Austria     47.85      11.41
3       Graz  Austria     47.08        NaN
4       Linz  Austria     48.32      14.29
